In [27]:
import constantes as cons
import sys
import pandas as pd
from backtesting import Backtest, Strategy
import pandas_ta as ta
import talib
import numpy as np

symbol='STORJUSDT'
mult_take_profit= 1
mult_stop_loss= 3

def get_bollinger_bands(df):
    mult = 2.0
    length = 20
    # calcular indicadores
    close = df['Close']
    basis = talib.SMA(close, length)
    dev = mult * talib.STDDEV(close, length)
    df['upper'] = basis + dev
    df['lower'] = basis - dev
    # imprimir resultados
    return df

def obtiene_historial(symbol):
    client = cons.client
    timeframe='30m'
    leido=False
    while leido==False:
        try:
            historical_data = client.get_historical_klines(symbol, timeframe)
            leido = True
        except KeyboardInterrupt as ky:
            print("\nSalida solicitada. ")
            sys.exit()              
        except:
            print("Intento leer de nuevo...")
            pass
    data = pd.DataFrame(historical_data)
    data.columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 
                        'Number of Trades', 'TB Base Volume', 'TB Quote Volume', 'Ignore']
    data['Open Time'] = pd.to_datetime(data['Open Time']/1000, unit='s')
    data['Close Time'] = pd.to_datetime(data['Close Time']/1000, unit='s')
    numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote Asset Volume', 'TB Base Volume', 'TB Quote Volume']
    data[numeric_columns] = data[numeric_columns].apply(pd.to_numeric, axis=1)
    data['timestamp']=data['Open Time']
    data.set_index('timestamp', inplace=True)
    data.dropna(inplace=True)
    data.drop(['Open Time','Close Time','Quote Asset Volume', 'TB Base Volume', 'TB Quote Volume','Number of Trades',
            'Ignore'], axis=1, inplace=True)    
    data['ema20']=data.ta.ema(20)
    data['ema50']=data.ta.ema(50)
    data['ema200']=data.ta.ema(200)
    data['atr']=ta.atr(data.High, data.Low, data.Close)
    data = get_bollinger_bands(data)
    return data

def backtesting_validation(data,print_output=False): 
    class Predictor2(Strategy):
        def init(self):
            pass    
        def next(self):
            if not self.position:
                if self.data.signal[-1] ==1:
                    self.buy(size=100, sl=self.data.stop_loss[-1] , tp=self.data.take_profit[-1])
                elif self.data.signal[-1] ==-1:
                    self.sell(size=100, sl=self.data.stop_loss[-1] , tp=self.data.take_profit[-1])
            else:
                pass
    bt = Backtest(data, Predictor2, cash=1000, commission=.002, exclusive_orders=True)
    output = bt.run()
    if print_output:
        print(output)

def estrategia(data):
    data['signal']=  np.where( (data.ema20 > data.ema50) & (data.ema50 > data.ema200) ,1,
                    (np.where( (data.ema20 < data.ema50) & (data.ema50 < data.ema200) ,-1,
                            0)))        
    data['take_profit'] =np.where(data.signal==1,data.Close+(data.atr*mult_take_profit) ,np.where(data.signal==-1,data.Close-(data.atr*mult_take_profit),0))
    data['stop_loss']   =np.where(data.signal==1,data.Close-(data.atr*mult_stop_loss)   ,np.where(data.signal==-1,data.Close+(data.atr*mult_stop_loss),0))
    return data

data = obtiene_historial(symbol)
data=estrategia(data)
data

,Open,High,Low,Close,Volume,ema20,ema50,ema200,atr,upper,lower,signal,take_profit,stop_loss
timestamp,,,,,,,,,,,,,,
2023-06-04 00:00:00,0.2995,0.2995,0.2984,0.2985,27586.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0.000000,0.000000
2023-06-04 00:30:00,0.2985,0.2985,0.2968,0.2968,36227.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0.000000,0.000000
2023-06-04 01:00:00,0.2969,0.2984,0.2963,0.2983,46124.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0.000000,0.000000
2023-06-04 01:30:00,0.2986,0.2990,0.2985,0.2990,12803.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0.000000,0.000000
2023-06-04 02:00:00,0.2987,0.2995,0.2987,0.2989,3906.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-24 17:30:00,0.2717,0.2736,0.2692,0.2730,283419.0,0.271238,0.268608,0.256670,0.003460,0.276391,0.267599,1,0.276460,0.262620
2023-06-24 18:00:00,0.2732,0.2742,0.2719,0.2729,169426.0,0.271397,0.268776,0.256832,0.003377,0.276477,0.267683,1,0.276277,0.262768
2023-06-24 18:30:00,0.2723,0.2736,0.2720,0.2731,43951.0,0.271559,0.268946,0.256994,0.003250,0.276422,0.267688,1,0.276350,0.263349


In [25]:
data